In [ ]:
%load_ext autoreload
%autoreload 2

# Deployment

> Putting a model into Deployment

In [ ]:
#| default_exp deployment

## Initial Checks 

In [ ]:
#|eval: false
!conda list | grep "pytorch"

ffmpeg                    4.3                  hf484d3e_0    pytorch
pytorch                   2.0.1           py3.11_cuda11.8_cudnn8.7.0_0    pytorch
pytorch-cuda              11.8                 h7e8668a_5    pytorch
pytorch-ignite            0.4.12                   pypi_0    pypi
pytorch-lightning         2.0.7                    pypi_0    pypi
pytorch-mutex             1.0                        cuda    pytorch
torchaudio                2.0.2               py311_cu118    pytorch
torchtriton               2.0.0                     py311    pytorch
torchvision               0.15.2              py311_cu118    pytorch


In [ ]:
#|eval: false
!nvcc --version

/bin/bash: line 1: nvcc: command not found


In [ ]:
#|eval: false
!pip list | grep "fastai" 
!pip list | grep "fastbook"
!pip list | grep "ipywidgets"

^C
ERROR: Operation cancelled by user
fastbook                  0.0.29
ipywidgets                7.8.0


In [ ]:
#|eval: false
import torch

In [ ]:
#|eval: false
torch.cuda.is_available()

True

## Gather Data

In [ ]:
#|eval: false
from fastbook import search_images_ddg
from fastdownload import download_url
from fastai.vision.all import *
from fastAIcourse.functions import *
import os

In [ ]:
#|eval: false
dest = 'grizzly'
download_pic(dest)

In [ ]:
#|code-fold: true
#|eval: false
searches = ('grizzly bears','black bears','teddy bears')
path = Path('bears')

create_data_folder(path, searches)

## Data Augmentation and Designing model

In [ ]:
#|eval: false
?DataBlock

In [ ]:
#|eval: false
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
#|eval: false
dls = bears.dataloaders(path)

In [ ]:
#|eval: false
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#|eval: false
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#|eval: false
bears = bears.new(
    item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#|eval: false
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

## Training Your Model, and Using It to Clean Your Data

In [ ]:
#|eval: false
bears = bears.new(
    item_tfms=RandomResizedCrop(128, min_scale=0.5),
    batch_tfms=aug_transforms(mult=1))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=False)

In [ ]:
#|eval: false
dls.valid.show_batch(max_n=8, nrows=2, unique=False)

## Training Your Model

In [ ]:
#|eval: false
import timm 

In [ ]:
#|eval: false
timm.list_models('resnet1*')[0:5]

In [ ]:
#|eval: false
learn = vision_learner(dls,  'resnet18', metrics=error_rate)
learn.fine_tune(5)

In [ ]:
#|eval: false
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#|eval: false
interp.plot_top_losses(6, nrows=2)

In [ ]:
#|eval: false
from fastai.vision.widgets import ImageClassifierCleaner

In [ ]:
#|eval: false
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#hide
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

## Testing

In [ ]:
#|eval: false
searches = ('grizzly bears','black bears','teddy bears')
image= 'grizzly0.jpg'


is_real,_,probs = learn.predict(PILImage.create(image))

for index, value in enumerate(searches):
    np.set_printoptions(suppress=True, precision=4)
    print(f"Probability of {value} is :{probs[index]:.5f}.")
    
max_value, max_index = torch.max(probs, dim=0)
print(f"This is a: {searches[max_index]} with probability: {max_value:.5f}.")


Image.open(image).to_thumb(256,256)

## Turning Your Model into an Online Application

In [ ]:
#|eval: false
learn.export('model.pkl')

In [ ]:
#|eval: false
path = Path()
filename = path.ls(file_exts='.pkl')
filename[0].name

In [ ]:
#|eval: false
learn_inf = load_learner(path/filename[0].name)

In [ ]:
#|eval: false
%time learn_inf.predict('grizzly0.jpg')

In [ ]:
#|eval: false
learn_inf.dls.vocab

## Making a widget application

In [ ]:
!pip list | grep 'ipywidgets'

In [ ]:
#|eval: false
import ipywidgets as widgets

In [ ]:
#|eval: false
btn_upload = widgets.FileUpload(
    button_style='success',
    description='Upload',
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
display(btn_upload)


In [ ]:
#|eval: false

if btn_upload.value:
    uploaded_image = Image.open(io.BytesIO(btn_upload.value[-1].content)).to_thumb(256,256)
    display(uploaded_image)

In [ ]:
#|eval: false
out_pl = widgets.Output()
out_pl

In [ ]:
#|eval: false
if btn_upload.value:
    out_pl = widgets.Output()
    out_pl.clear_output()
    with out_pl: display(uploaded_image)
    out_pl


In [ ]:
#|eval: false
if btn_upload.value:
    uploaded_image = Image.open(io.BytesIO(btn_upload.value[-1].content)).to_thumb(256,256)
    pred,pred_idx,probs = learn_inf.predict(uploaded_image)


In [ ]:
#|eval: false
#hide_output
if btn_upload.value:
    lbl_pred = widgets.Label()
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    display(lbl_pred)


In [ ]:
#|eval: false
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
#|eval: false
def on_click_classify(change):
    if btn_upload.value:
        img = Image.open(io.BytesIO(btn_upload.value[-1].content)).to_thumb(256,256)
        out_pl.clear_output()
        with out_pl: display(img)
        pred,pred_idx,probs = learn_inf.predict(img)
        lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    else:
        lbl_pred.value = f'Select image'

btn_run.on_click(on_click_classify)

In [ ]:
#|eval: false
if btn_upload.value:
    lbl_pred

In [ ]:
#|eval: false
if btn_upload.value:
    screen = widgets.VBox([widgets.Label('Select your bear!'), 
                  btn_upload,
                  btn_run,
                  out_pl, 
                  lbl_pred]
                )
    display(screen)